# Machine Learning for String Field Theory

H. Erbin, R. Finotello, M. Kudrna, M. Schnabl

---
---

## Abstract

In the framework of bosonic Open String Field Theory (OSFT), we consider several observables characterised by conformal weight and type, and the position of vacua in the potential for various values of truncated mass level. We focus on the prediction of the extrapolated value for the level-$\infty$ truncation using Machine Learning (ML) techniques.

## Synopsis

In this notebook we prepare the model independent analysis. In particular we study a way to reduce the input data in order to use them independently from the initial model.

In [ ]:
%load_ext autoreload
%autoreload 2

## Load the Dataset

In [2]:
import pandas as pd

lumps  = pd.read_csv('./data/lumps.csv')
wzw    = pd.read_csv('./data/wzw.csv')

In [3]:
import re

dlumps = pd.read_json('./data/dlumps.json')
dexp   = pd.read_json('./data/dlumps_exp.json')

dlumps = pd.merge(dlumps, dexp, left_index=True, right_index=True)
dlumps = dlumps.rename(columns=lambda c: re.sub('^([0-9]+)', r'level_\1', c))

In [4]:
lumps.describe()

,solutions,init,weight,type,exp,level_2,level_3,level_4,level_5,level_6,...,level_9,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18
count,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,...,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,732.000000,7.320000e+02
mean,23.849727,0.832923,1.983402,3.748634,0.538251,-1.605744,-1.770120,7.840907,8.296700,-33.868139,...,176.059381,-737.888108,-761.659480,3002.674618,3087.526701,-11341.308600,-11628.506465,39852.334515,40771.690931,-1.309313e+05
std,13.322038,0.997060,2.338717,0.663437,0.707522,4.515971,4.969205,21.374749,22.621096,110.076428,...,630.770974,2826.575772,2915.937396,11805.371641,12130.574351,44982.958696,46093.166999,158645.302316,162223.144544,5.230833e+05
min,0.000000,0.000000,0.000000,2.000000,-1.000000,-19.744040,-21.893983,-0.754568,-0.782633,-514.984097,...,-1.781985,-13321.170445,-13781.246472,-8.850113,-12.265769,-211473.396816,-216475.644423,-44.356923,-66.596211,-2.489024e+06
25%,12.750000,0.000000,0.158447,4.000000,0.000000,-0.776018,-1.031007,0.000000,0.000000,-0.919728,...,0.000000,-1.106747,-1.780883,0.001452,0.001633,-2.764240,-5.863000,0.041082,0.040815,-1.850164e+01
50%,25.000000,0.000000,1.000000,4.000000,1.000000,0.000000,0.000000,0.924598,0.935917,0.000000,...,0.975852,0.001638,0.001847,0.987451,0.987109,0.001680,0.018444,1.001152,1.000000,1.859456e-01
75%,35.250000,1.650000,3.226174,4.000000,1.000000,0.817407,0.917998,1.387335,1.524975,0.965868,...,2.129576,0.993199,0.998879,4.057912,6.077472,0.999664,1.000738,14.953725,13.923463,1.000880e+00
max,45.000000,3.000000,9.000000,4.000000,1.000000,1.239384,1.358098,122.931347,131.675490,2.275741,...,2890.129013,5.243298,6.283092,56115.100219,57592.698860,16.106978,23.077325,731718.332090,748286.961169,1.033588e+02


In [5]:
dlumps.describe()

,init,weight,type,level_2,level_3,level_4,level_5,level_6,level_7,level_8,...,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,exp
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01
mean,1.050000,1.715278,3.800000,-2.608983,-3.087235,12.360494,13.485368,-53.858729,-57.641226,258.016894,...,-1172.908731,-1232.022585,5292.641517,5529.478551,-22317.746351,-23199.704152,8.566285e+04,8.869707e+04,-3.006011e+05,-7.239206e+05
std,1.468081,2.189245,0.615587,8.118042,9.802525,44.393799,48.702023,222.478514,238.231213,1116.816835,...,5174.917644,5439.189243,23531.629810,24590.746496,99564.849566,103507.763796,3.826730e+05,3.962378e+05,1.343627e+06,3.237079e+06
min,0.000000,0.000000,2.000000,-33.620593,-40.373713,-1.734322,-1.825242,-997.122136,-1067.678440,-5.550577,...,-23157.212916,-24338.986279,-7.423880,-17.395405,-445320.847252,-462954.335124,-6.706599e+01,-6.704860e+01,-6.009038e+06,-1.447675e+07
25%,0.000000,0.090278,4.000000,-1.304863,-1.425327,0.000000,0.000000,-1.437197,-1.412107,-0.068770,...,-1.958771,-1.390445,-0.195941,-0.652932,-1.970593,-1.375239,-6.939012e-01,-1.802527e+00,-8.913318e+00,-1.995897e+00
50%,0.000000,1.000000,4.000000,0.000000,0.000000,0.011280,0.001635,0.000000,0.000000,0.010817,...,0.000000,0.000272,1.219497,0.424945,0.000099,0.201399,1.182381e+00,4.360167e-01,2.550822e-01,3.300225e-01
75%,3.000000,2.381944,4.000000,0.006879,0.341444,2.107497,2.043995,0.623679,0.639251,2.023673,...,1.222807,1.711390,2.185291,2.014058,1.580704,2.010701,6.847626e+00,2.034738e+00,2.008174e+00,1.998589e+00
max,3.000000,9.000000,4.000000,2.171332,2.499696,198.953725,218.118294,2.036695,2.032430,5001.157827,...,3.761426,9.229395,105266.161459,110002.605965,23.210856,23.159297,1.711459e+06,1.772124e+06,1.469126e+02,1.172284e+02


In [6]:
wzw.describe()

,k,exp_re,exp_im,weight,j,m,type,level_2_re,level_2_im,level_3_re,...,level_6_re,level_6_im,level_7_re,level_7_im,level_8_re,level_8_im,level_9_re,level_9_im,level_10_re,level_10_im
count,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,...,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000,1647.000000
mean,6.784457,0.039473,-0.001890,0.904660,2.002429,0.000000,3.888282,-0.039372,-0.005783,-0.038920,...,0.002130,-0.002027,0.010572,-0.015218,0.073137,-0.011159,0.065939,0.009917,0.000521,0.006889
std,1.309949,0.596876,0.301625,0.594117,1.252577,1.587753,0.459439,2.536976,0.237263,2.571754,...,1.391057,0.372175,1.443925,0.448329,1.531316,0.476229,1.591242,0.629464,1.738222,0.663840
min,2.000000,-1.519671,-0.930605,0.000000,0.000000,-4.000000,2.000000,-26.284377,-1.867961,-26.284377,...,-10.978029,-2.287981,-11.445648,-3.646223,-13.721069,-4.023181,-14.249796,-6.650984,-24.994666,-7.174097
25%,6.000000,-0.441788,0.000000,0.375000,1.000000,-1.000000,4.000000,-0.558114,0.000000,-0.579659,...,-0.521596,0.000000,-0.530032,0.000000,-0.498717,0.000000,-0.486161,0.000000,-0.478383,0.000000
50%,7.000000,0.000000,0.000000,0.972222,2.000000,0.000000,4.000000,0.000476,0.000000,0.004674,...,0.000000,0.000000,0.000000,0.000000,0.009156,0.000000,0.006125,0.000000,0.000000,0.000000
75%,8.000000,0.506705,0.000000,1.333333,3.000000,1.000000,4.000000,0.600366,0.000000,0.633898,...,0.545093,0.000000,0.564691,0.000000,0.596880,0.000000,0.589068,0.000000,0.521422,0.000000
max,8.000000,1.414214,0.930605,2.000000,4.000000,4.000000,4.000000,35.385221,1.867961,35.385221,...,10.978029,2.287981,11.445648,3.646223,18.673134,4.023181,20.051284,6.683011,15.315423,7.187127


The main difference between the lump solutions and the WZW model is the fact that truncation levels are complex numbers in the latter.
Moreover solutions in the WZW are labelled by an integer `k` which is related to the `weight` variable as `weight` $= \frac{j(j+1)}{k+2}$ where $j$ is one of the quantum numbers associated to the $\mathrm{SU}(2)$ representation, together with $m$.

## Preparing the Input

In [7]:
wzw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   k            1647 non-null   float64
 1   exp_re       1647 non-null   float64
 2   exp_im       1647 non-null   float64
 3   weight       1647 non-null   float64
 4   j            1647 non-null   float64
 5   m            1647 non-null   float64
 6   type         1647 non-null   float64
 7   level_2_re   1647 non-null   float64
 8   level_2_im   1647 non-null   float64
 9   level_3_re   1647 non-null   float64
 10  level_3_im   1647 non-null   float64
 11  level_4_re   1647 non-null   float64
 12  level_4_im   1647 non-null   float64
 13  level_5_re   1647 non-null   float64
 14  level_5_im   1647 non-null   float64
 15  level_6_re   1647 non-null   float64
 16  level_6_im   1647 non-null   float64
 17  level_7_re   1647 non-null   float64
 18  level_7_im   1647 non-null   float64
 19  level_

We also have to redefine the columns of the lump and double lump solutions to have complex entries:

In [8]:
import numpy as np
import re

zeros  = np.zeros(lumps.shape[0], dtype=int)
dzeros = np.zeros(dlumps.shape[0], dtype=int)
lumps  = lumps.filter(regex='^weight|^type|^exp|^level')
dlumps = dlumps.filter(regex='^weight|^type|^exp|^level')

# rename columns
rename = {c: c + '_re' for c in lumps.columns if re.match('^level.*|^exp', c)}
lumps  = lumps.rename(columns=rename)
dlumps = dlumps.rename(columns=rename)

# add columns 
cols = [re.sub('_re', '_im', c) for c in lumps.columns if re.match('.*_re$', c)]
for c in cols:
    lumps[c]  = zeros
    dlumps[c] = dzeros

In [9]:
lumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 38 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   weight       732 non-null    float64
 1   type         732 non-null    int64  
 2   exp_re       732 non-null    int64  
 3   level_2_re   732 non-null    float64
 4   level_3_re   732 non-null    float64
 5   level_4_re   732 non-null    float64
 6   level_5_re   732 non-null    float64
 7   level_6_re   732 non-null    float64
 8   level_7_re   732 non-null    float64
 9   level_8_re   732 non-null    float64
 10  level_9_re   732 non-null    float64
 11  level_10_re  732 non-null    float64
 12  level_11_re  732 non-null    float64
 13  level_12_re  732 non-null    float64
 14  level_13_re  732 non-null    float64
 15  level_14_re  732 non-null    float64
 16  level_15_re  732 non-null    float64
 17  level_16_re  732 non-null    float64
 18  level_17_re  732 non-null    float64
 19  level_18

In [10]:
dlumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 38 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   weight       20 non-null     float64
 1   type         20 non-null     int64  
 2   level_2_re   20 non-null     float64
 3   level_3_re   20 non-null     float64
 4   level_4_re   20 non-null     float64
 5   level_5_re   20 non-null     float64
 6   level_6_re   20 non-null     float64
 7   level_7_re   20 non-null     float64
 8   level_8_re   20 non-null     float64
 9   level_9_re   20 non-null     float64
 10  level_10_re  20 non-null     float64
 11  level_11_re  20 non-null     float64
 12  level_12_re  20 non-null     float64
 13  level_13_re  20 non-null     float64
 14  level_14_re  20 non-null     float64
 15  level_15_re  20 non-null     float64
 16  level_16_re  20 non-null     float64
 17  level_17_re  20 non-null     float64
 18  level_18_re  20 non-null     float64
 19  exp_re    

## Principal Components

We then need to compute the principal components of the truncation levels since we are interested in having the same input shape.

In [11]:
from sklearn.decomposition import PCA

# fix random state
RAND = 123
COMP = 10

# compute PCA
lumps_levels  = PCA(n_components=COMP, random_state=RAND).fit_transform(lumps.filter(regex='^level'))
dlumps_levels = PCA(n_components=COMP, random_state=RAND).fit_transform(dlumps.filter(regex='^level'))
wzw_levels    = PCA(n_components=COMP, random_state=RAND).fit_transform(wzw.filter(regex='^level'))

In [12]:
import pandas as pd

# recreate a dataframe
cols = ['pca_' + str(n+1) for n in range(COMP)]
lumps_levels  = pd.DataFrame(lumps_levels, columns=cols)
dlumps_levels = pd.DataFrame(dlumps_levels, columns=cols)
wzw_levels    = pd.DataFrame(wzw_levels, columns=cols)

In [13]:
# join the old dataframes
lumps  = pd.merge(lumps[['exp_re', 'exp_im', 'weight', 'type']], lumps_levels, left_index=True, right_index=True)
dlumps = pd.merge(dlumps[['exp_re', 'exp_im', 'weight', 'type']], dlumps_levels, left_index=True, right_index=True)
wzw    = pd.merge(wzw[['exp_re', 'exp_im', 'weight', 'type']], wzw_levels, left_index=True, right_index=True)

We finally need to join (_outer_ join) the datasets:

In [14]:
# convert all integer to floats
lumps  = lumps.astype(float)
dlumps = dlumps.astype(float)
wzw    = wzw.astype(float)

# merge the datasets (keep the double lumps apart)
df = pd.concat([lumps, wzw], axis=0)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2379 entries, 0 to 1646
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   exp_re  2379 non-null   float64
 1   exp_im  2379 non-null   float64
 2   weight  2379 non-null   float64
 3   type    2379 non-null   float64
 4   pca_1   2379 non-null   float64
 5   pca_2   2379 non-null   float64
 6   pca_3   2379 non-null   float64
 7   pca_4   2379 non-null   float64
 8   pca_5   2379 non-null   float64
 9   pca_6   2379 non-null   float64
 10  pca_7   2379 non-null   float64
 11  pca_8   2379 non-null   float64
 12  pca_9   2379 non-null   float64
 13  pca_10  2379 non-null   float64
dtypes: float64(14)
memory usage: 278.8 KB


In [16]:
dlumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   exp_re  20 non-null     float64
 1   exp_im  20 non-null     float64
 2   weight  20 non-null     float64
 3   type    20 non-null     float64
 4   pca_1   20 non-null     float64
 5   pca_2   20 non-null     float64
 6   pca_3   20 non-null     float64
 7   pca_4   20 non-null     float64
 8   pca_5   20 non-null     float64
 9   pca_6   20 non-null     float64
 10  pca_7   20 non-null     float64
 11  pca_8   20 non-null     float64
 12  pca_9   20 non-null     float64
 13  pca_10  20 non-null     float64
dtypes: float64(14)
memory usage: 2.3 KB


In [17]:
# save the dataset
df.to_csv('./data/full.csv', index=False)
dlumps.to_csv('./data/dlumps.csv', index=False)